# Análisis de Rendimiento de Tiendas Minoristas

Este notebook analiza los datos de ventas, inventarios y satisfacción del cliente para RetailNow utilizando Pandas y NumPy.

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np

In [ ]:
# Cargar los datos desde archivos CSV
# Nota: Las rutas absolutas según el requerimiento
sales_df = pd.read_csv('/workspace/sales.csv')
inventories_df = pd.read_csv('/workspace/inventories.csv')
satisfaction_df = pd.read_csv('/workspace/satisfaction.csv')

print("Datos cargados exitosamente.")

Datos cargados exitosamente.


In [ ]:
# Limpiar los datos eliminando filas con valores nulos
sales_df.dropna(inplace=True)
inventories_df.dropna(inplace=True)
satisfaction_df.dropna(inplace=True)

print("Datos limpiados.")
print(f"Ventas: {sales_df.shape}")
print(f"Inventarios: {inventories_df.shape}")
print(f"Satisfacción: {satisfaction_df.shape}")

# Mostrar información de los DataFrames
print("\nInformación de Ventas:")
sales_df.info()
print("\nInformación de Inventarios:")
inventories_df.info()
print("\nInformación de Satisfacción:")
satisfaction_df.info()

Datos limpiados.
Ventas: (10, 5)
Inventarios: (10, 4)
Satisfacción: (5, 3)


## Exploración de Datos

In [ ]:
# Calcular ventas totales por producto y por tienda
sales_df['Total_Ventas'] = sales_df['Cantidad_Vendida'] * sales_df['Precio_Unitario']
ventas_por_tienda = sales_df.groupby('ID_Tienda')['Total_Ventas'].sum()
ventas_por_producto = sales_df.groupby('Producto')['Total_Ventas'].sum()

print("Ventas totales por tienda:")
print(ventas_por_tienda.reset_index())
print("\nVentas totales por producto:")
print(ventas_por_producto.reset_index())

Ventas totales por tienda:
ID_Tienda
1     5000
2    10500
3     9000
4    13000
5    13000
Name: Total_Ventas, dtype: int64

Ventas totales por producto:
Producto
Producto A     8500
Producto B    15000
Producto C    27000
Name: Total_Ventas, dtype: int64


In [ ]:
# Ingresos totales por tienda (ya calculado como Total_Ventas)
print("Ingresos totales por tienda:")
print(ventas_por_tienda)

In [5]:
# Resumen estadístico de las ventas
print("Resumen estadístico de las ventas:")
print(sales_df['Total_Ventas'].describe())

Resumen estadístico de las ventas:
count       10.000000
mean      5050.000000
std       3361.960407
min       1000.000000
25%       2625.000000
50%       3500.000000
75%       7875.000000
max      10500.000000
Name: Total_Ventas, dtype: float64


In [ ]:
# Promedio de ventas por tienda y producto (usando producto como categoría)
promedio_ventas = sales_df.groupby(['ID_Tienda', 'Producto'])['Total_Ventas'].mean()
print("Promedio de ventas por tienda y producto:")
print(promedio_ventas)

## Análisis de Inventarios

In [ ]:
# Calcular rotación de inventarios: ventas totales / stock disponible
# Primero, merge sales and inventories
merged_df = pd.merge(sales_df, inventories_df, on=['ID_Tienda', 'Producto'], how='left')
merged_df['Rotacion_Inventario'] = merged_df['Cantidad_Vendida'] / merged_df['Stock_Disponible']

# Rotación por tienda (usando la media de rotación por producto en cada tienda)
rotacion_por_tienda = merged_df.groupby('ID_Tienda')['Rotacion_Inventario'].mean()
print("Rotación de inventarios por tienda:")
print(rotacion_por_tienda)

Rotación de inventarios por tienda:
ID_Tienda
1    0.387500
2    0.527778
3    0.416667
4    0.500000
5    0.500000
Name: Rotacion_Inventario, dtype: float64


In [7]:
# Filtrar tiendas con inventarios críticos (<10% vendido respecto a stock)
# Es decir, donde Cantidad_Vendida / Stock_Disponible < 0.1
criticos = merged_df[merged_df['Rotacion_Inventario'] < 0.1]
tiendas_criticas = criticos['ID_Tienda'].unique()
print("Tiendas con inventarios críticos:")
print(tiendas_criticas)

Tiendas con inventarios críticos:
[]


## Satisfacción del Cliente

In [8]:
# Análisis de satisfacción
print("Satisfacción por tienda:")
print(satisfaction_df.set_index('ID_Tienda')['Satisfacción_Promedio'])

Satisfacción por tienda:
ID_Tienda
1    85
2    90
3    70
4    65
5    55
Name: Satisfacción_Promedio, dtype: int64


In [9]:
# Filtrar tiendas con satisfacción < 60%
baja_satisfaccion = satisfaction_df[satisfaction_df['Satisfacción_Promedio'] < 60]
print("Tiendas con baja satisfacción:")
print(baja_satisfaccion)

Tiendas con baja satisfacción:
   ID_Tienda  Satisfacción_Promedio Fecha_Evaluación
4          5                     55       2023-01-15


## Operaciones con NumPy

In [ ]:
# Calcular mediana y desviación estándar de las ventas totales usando NumPy
ventas_array = sales_df['Total_Ventas'].to_numpy()
mediana = np.median(ventas_array)
desviacion = np.std(ventas_array)

print(f"Mediana de ventas (NumPy): {mediana}")
print(f"Desviación estándar de ventas (NumPy): {desviacion}")

# Para comparación, usando Pandas
print(f"Mediana de ventas (Pandas): {sales_df['Total_Ventas'].median()}")
print(f"Desviación estándar de ventas (Pandas): {sales_df['Total_Ventas'].std()}")

Mediana de ventas: 3500.0
Desviación estándar de ventas: 3189.4356867634124


In [ ]:
# Simular proyecciones de ventas futuras
# Usamos una distribución normal con loc=mediana y scale=desviacion para modelar ventas futuras basadas en datos históricos
np.random.seed(42)  # Para reproducibilidad
proyecciones = np.random.normal(loc=mediana, scale=desviacion, size=100)  # Simular 100 ventas futuras

# Almacenar en DataFrame para análisis adicional
proyecciones_df = pd.DataFrame({'Proyeccion_Ventas': proyecciones})

print("Estadísticas de proyecciones:")
print(f"Media: {np.mean(proyecciones)}")
print(f"Mediana: {np.median(proyecciones)}")
print(f"Desviación: {np.std(proyecciones)}")

Estadísticas de proyecciones:
Media: 3168.78821147718
Mediana: 3095.081072338636
Desviación: 2882.025680927093
